# Community models - pFBA predictions

In [28]:
import reframed
import pandas as pd
import numpy as np

import seaborn as sns

import os
import json

import functions.translation_dicts as translation_dicts

In [29]:
compounds_dict, source_dict,substrate_dict, gas_sheet_dict, community_dict = translation_dicts.translation_dicts()

#### Load data

In [14]:
all_mags_paper = pd.read_excel("input/files_from_fairdomhub/All_Mags_for_paper_analysis.xlsx",sheet_name="Coverage")
all_mags_paper.set_index("MAG",inplace=True)
all_mags_paper.head(2)

,Source,Substrate,Completeness (%),Contamination (%),Domain,Phylum,Class,Order,Family,Genus,Species,Genome size (bp),Scaffolds,N50,CDS prediction (DRAM),Coverage (%)
MAG,,,,,,,,,,,,,,,,
CH3-bin.2,Marshland,Xylan,99.19,1.61,Bacteria,Firmicutes_A,Clostridia,Clostridiales,Clostridiaceae,Clostridium,Clostridium sp000230835,6175525,63,230012,5840,74.31
CH9-bin.0,Cow_Manure,Xylan,99.03,2.82,Bacteria,Actinobacteriota,Actinomycetia,Actinomycetales,Bifidobacteriaceae,Bifidobacterium,NaN,2513253,38,128964,2055,65.10


#### Load models

In [15]:
with open("output/GEMs/GEMs_adapt.json") as json_file:
    GEMs_adapt = json.load(json_file)
    
gems_adapt_dict = {"no_constr":"output/GEMs/GEMs_no_constraints/",
                  "constr0_1":"output/GEMs/GEMs_soft_constraints_score_0.1/"}


In [16]:
with open("output/soft_constraints_v2/relevant_MAGs_99.txt") as text_file:
    relevant_MAGs = text_file.read().split("\n")

relevant_MAGs = [string.replace("\t","") for string in relevant_MAGs]


In [17]:
GEMs_dict = {}

directory = os.fsencode("output/GEMs/GEMs_adapt_media_ACt2r/")

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    print(filename)

    GEMs_dict[filename[:-4]]= reframed.load_cbmodel("output/GEMs/GEMs_adapt_media_ACt2r/"+filename)



CH3-bin.2
CH9-bin.0
CH14-bin.1
CH1-bin.6
CH7-bin.23
CH8-bin.22
CH13-bin.12
CH1-bin.9
CH3-bin.1
CH14-bin.2
CH15-bin.0
CH13-bin.14
CH9-bin.6
CH13-bin.4
CH15-bin.1
CH15-bin.23
CH8-bin.17
CH13-bin.11
CH15-bin.7
CH7-bin.16
CH15-bin.16
CH15-bin.8
CH13-bin.25
CH7-bin.11
CH8-bin.25
CH9-bin.1
CH7-bin.13
CH8-bin.2
CH15-bin.17
CH15-bin.22
CH1-bin.8
CH15-bin.6
CH8-bin.6
CH13-bin.0
CH8-bin.7
CH13-bin.17
CH1-bin.3
CH15-bin.5
CH15-bin.13
CH7-bin.8
CH1-bin.4
CH7-bin.18
CH8-bin.5
CH9-bin.2
CH7-bin.2
CH9-bin.5
CH7-bin.4
CH8-bin.8
CH15-bin.10
CH7-bin.1
CH15-bin.12
CH7-bin.17
CH9-bin.4
CH1-bin.7
CH15-bin.15
CH8-bin.16
CH8-bin.9
CH7-bin.12
CH13-bin.1
CH7-bin.20
CH8-bin.21
CH8-bin.14
CH7-bin.9
CH13-bin.13
CH13-bin.2
CH8-bin.4
CH15-bin.18


#### Community members

In [18]:
MAG2community_id = pd.read_csv("output/soft_constraints/MAG2sour_sub_id.tsv",sep="\t",header=None)
MAG2community_id.set_index(0,inplace=True)

### Merge community for each experiment

- Define communities: groupby source and substrate

In [19]:
communities = MAG2community_id[MAG2community_id.index.isin(relevant_MAGs)].groupby(1).groups

In [20]:
community_models = {}

for community_id, MAGs in communities.items():
    models = []
    for MAG in MAGs:
        models.append(GEMs_dict[MAG])

    community_models[community_id] = reframed.Community(community_id,models)

### Define environment

**From SynCon2 to SynCon1**

"SynCon2 had the same composition, but it lacked ascorbic acid, and it had two additional trace elements (selenium and wolfram)"

Add
- Ascorbate: ascb__L

Remove
- Selenite (inorganic selenium): slnt
- (wolfram: not in BiGG database)

In [21]:
syncon2 = pd.read_csv("input/syncon2media_combined.csv",header=None)

syncon1 = list(syncon2[0].values)

syncon1.remove("slnt")

syncon1.append("ascb__L")

In [22]:
env_syncon1 = reframed.Environment.from_compounds(syncon1)

**Specific environments**

Find how many mols of a compound you would need to have the same amount of carbon as one mol of glucose. 

Strategy: 
- Load universal model
- For each substrate:
    - Find number of carbon atoms in the compound. n_comp_0 = 6 (=#C in glucose)/#C in compound
    - Set the final amount as the uptake rate

In [23]:
model_uni = reframed.load_cbmodel("/Users/idunmariaburgos/universal_model_extension/output/universe_bacteria.xml")

In [24]:
substrate_composition = {"Avicel":["cellb","cell3","cell4","cell5"],
                         "PASC":["cellb","cell3","cell4","cell5"],
                         "Xylan":["xylb","xyl3","xylan4","xylan8"]}

In [25]:
import copy
import re

In [26]:
substrate_mols = {}

for substrate_name, composition in substrate_composition.items():
    substrate_mols[substrate_name] = {}

    
    for met in composition:
        
        formula = copy.copy(model_uni.metabolites["M_"+met+"_e"].metadata["FORMULA"])
        list_atoms = re.split("(?<=[0-9])(?=[a-zA-Z])",formula)
        carbon = int(list_atoms[0][1:])
        
        substrate_mols[substrate_name][met]= 6/(carbon)

In [61]:
substrate_mols

{'Avicel': {'cellb': 0.5,
  'cell3': 0.3333333333333333,
  'cell4': 0.25,
  'cell5': 0.2},
 'PASC': {'cellb': 0.5,
  'cell3': 0.3333333333333333,
  'cell4': 0.25,
  'cell5': 0.2},
 'Xylan': {'xylb': 0.6,
  'xyl3': 0.4,
  'xylan4': 0.23076923076923078,
  'xylan8': 0.11538461538461539}}

In [31]:
syncon1_environments = {}
for substrate_name,met_mols in substrate_mols.items():
    syncon1_environments[substrate_name]= reframed.Environment.from_compounds(syncon1)
    for met,mol in met_mols.items():
        syncon1_environments[substrate_name]["R_EX_"+met+"_e"]=(-10,0)

In [32]:
syncon1_environments

{'Avicel': R_EX_zn2_e	-10.0	inf
 R_EX_mn2_e	-10.0	inf
 R_EX_fe3_e	-10.0	inf
 R_EX_k_e	-10.0	inf
 R_EX_b12_e	-10.0	inf
 R_EX_so4_e	-10.0	inf
 R_EX_ribflv_e	-10.0	inf
 R_EX_cl_e	-10.0	inf
 R_EX_nh4_e	-10.0	inf
 R_EX_btn_e	-10.0	inf
 R_EX_pydxn_e	-10.0	inf
 R_EX_ni2_e	-10.0	inf
 R_EX_rbflvrd_e	-10.0	inf
 R_EX_h2o_e	-10.0	inf
 R_EX_pi_e	-10.0	inf
 R_EX_fe2_e	-10.0	inf
 R_EX_fol_e	-10.0	inf
 R_EX_pnto__R_e	-10.0	inf
 R_EX_cu2_e	-10.0	inf
 R_EX_na1_e	-10.0	inf
 R_EX_tungs_e	-10.0	inf
 R_EX_mobd_e	-10.0	inf
 R_EX_cys__L_e	-10.0	inf
 R_EX_mg2_e	-10.0	inf
 R_EX_4abz_e	-10.0	inf
 R_EX_h_e	-10.0	inf
 R_EX_nac_e	-10.0	inf
 R_EX_ca2_e	-10.0	inf
 R_EX_cobalt2_e	-10.0	inf
 R_EX_thm_e	-10.0	inf
 R_EX_ascb__L_e	-10.0	inf
 R_EX_cellb_e	-10	0
 R_EX_cell3_e	-10	0
 R_EX_cell4_e	-10	0
 R_EX_cell5_e	-10	0,
 'PASC': R_EX_zn2_e	-10.0	inf
 R_EX_mn2_e	-10.0	inf
 R_EX_fe3_e	-10.0	inf
 R_EX_k_e	-10.0	inf
 R_EX_b12_e	-10.0	inf
 R_EX_so4_e	-10.0	inf
 R_EX_ribflv_e	-10.0	inf
 R_EX_cl_e	-10.0	inf
 R_EX_nh4_e	-10.0	inf

## Simulations

In [34]:
interesting_compounds = ["etoh","lac__L","ac","ppa","but","ibt","pta","hxa","isocap"]

In [35]:
model_comp_avicel = community_models["CD_A"].merged_model

syncon1_environments["Avicel"].apply(model_comp_avicel,inplace=True,exclusive=True)
constraints={}
constraints["R_EX_cys__L_e"]=0
constraints["R_EX_ascb__L_e"]=0

/Users/idunmariaburgos/opt/anaconda3/envs/MAGs_RecAndAn2/lib/python3.9/site-packages/reframed/core/environment.py:75: UserWarning: Exchange reaction not in model: R_EX_b12_e
  warn(f'Exchange reaction not in model: {r_id}')
/Users/idunmariaburgos/opt/anaconda3/envs/MAGs_RecAndAn2/lib/python3.9/site-packages/reframed/core/environment.py:75: UserWarning: Exchange reaction not in model: R_EX_rbflvrd_e
  warn(f'Exchange reaction not in model: {r_id}')
/Users/idunmariaburgos/opt/anaconda3/envs/MAGs_RecAndAn2/lib/python3.9/site-packages/reframed/core/environment.py:75: UserWarning: Exchange reaction not in model: R_EX_tungs_e
  warn(f'Exchange reaction not in model: {r_id}')


In [39]:
sol = reframed.pFBA(model_comp_avicel,objective={"R_EX_but_e":1},constraints=constraints)

In [41]:
sol.show_values(pattern="R_EX_",sort=True)

R_EX_cell4_e -10
R_EX_cellb_e -10
R_EX_cell5_e -10
R_EX_cell3_e -10
R_EX_h2o_e    102.962
R_EX_h_e      133.038
R_EX_co2_e    168
R_EX_but_e    168


In [42]:
sol = reframed.pFBA(model_comp_avicel,objective={"R_EX_hxa_e":1},constraints=constraints)

In [43]:
sol.show_values(pattern="R_EX_",sort=True)

R_EX_cell4_e -10
R_EX_cellb_e -10
R_EX_cell5_e -10
R_EX_cell3_e -10
R_EX_h_e      75
R_EX_hxa_e    105
R_EX_h2o_e    140
R_EX_co2_e    210


In [46]:
model_comp_xylan = community_models["CD_X"].merged_model
syncon1_environments["Xylan"].apply(model_comp_xylan,inplace=True,exclusive=True)

In [47]:
sol = reframed.pFBA(model_comp_xylan,objective={"R_EX_but_e":1},constraints=constraints)
sol.show_values(pattern="R_EX_",sort=True)

R_EX_h2o_e   -10
R_EX_so4_e   -10
R_EX_xylan4_e -10
R_EX_xylan8_e -10
R_EX_nh4_e   -10
R_EX_4hba_e   2.5
R_EX_gcald_e  2.5
R_EX_thm_e    2.5
R_EX_h2s_e    7.5
R_EX_h_e      8.75
R_EX_acald_e  22.5
R_EX_xyl__D_e  40.75
R_EX_but_e    51.25
R_EX_etoh_e   52.9167
R_EX_co2_e    167.917


In [48]:
sol = reframed.pFBA(model_comp_xylan,objective={"R_EX_hxa_e":1},constraints=constraints)
sol.show_values(pattern="R_EX_",sort=True)

R_EX_nh4_e   -10
R_EX_so4_e   -10
R_EX_xylan4_e -10
R_EX_xylan8_e -10
R_EX_4hba_e   2.5
R_EX_gcald_e  2.5
R_EX_thm_e    2.5
R_EX_acald_e  2.5
R_EX_h2s_e    7.5
R_EX_h_e      8.75
R_EX_etoh_e   15
R_EX_h2o_e    31.25
R_EX_xyl__D_e  42.75
R_EX_hxa_e    51.25
R_EX_co2_e    171.25


**Testing for all interesting compounds and all communities**

In [85]:
interesting_compounds = ["etoh","lac__L","ac","ppa","but","ibt","pta","ival","hxa","isocap"]

In [86]:
def find_key(value_entry):
    for key,value in substrate_dict.items():
        if value==value_entry:
            return key
    return None

In [87]:
pFBA_solutions = {}

#for community_id,community_model in example_communities.items():
for community_id,community_model in community_models.items():
    print(community_id)
    

    
    community_model_merged = community_model.merged_model
    
    # Apply polysaccharide specific media to model
    carbon_sub_name = find_key(community_id.split("_")[1])
    
    for carbon_sub_name, env in syncon1_environments.items():
        if carbon_sub_name=="Avicel":
            continue
        
        print("\t" + carbon_sub_name)
        
        pFBA_solutions[(community_id,carbon_sub_name)] = {}
        
        env.apply(community_model_merged,inplace=True,exclusive=True)

        # Shows carbon compounds used by community
        carbon_usable = [met for met in substrate_composition[carbon_sub_name] if "R_EX_"+met+"_e" in community_model_merged.get_exchange_reactions()]
        composition_count = len(carbon_usable)

        # Constrain model to take up the same amount of carbon as you get in 10 mmol/g*h glucose
        constraints = {"R_EX_"+met+"_e":-10*(substrate_mols[carbon_sub_name][met]/len(carbon_usable)) for met in carbon_usable}
        constraints["R_EX_cys__L_e"]=0
        constraints["R_EX_ascb__L_e"]=0

        pFBA_solutions[(community_id,carbon_sub_name)]["carbon_usable"]= carbon_usable


        for compound in interesting_compounds:
            print("\t\t" + compound)

            if "R_EX_"+compound+"_e" not in community_model_merged.get_exchange_reactions():
                    continue

            sol = reframed.pFBA(community_model_merged,objective={"R_EX_"+compound+"_e":1},constraints=constraints)

            pFBA_solutions[(community_id,carbon_sub_name)][compound]=sol.values["R_EX_"+compound+"_e"]

    

CD_A
	PASC
		etoh


/Users/idunmariaburgos/opt/anaconda3/envs/MAGs_RecAndAn2/lib/python3.9/site-packages/reframed/core/environment.py:75: UserWarning: Exchange reaction not in model: R_EX_b12_e
  warn(f'Exchange reaction not in model: {r_id}')
/Users/idunmariaburgos/opt/anaconda3/envs/MAGs_RecAndAn2/lib/python3.9/site-packages/reframed/core/environment.py:75: UserWarning: Exchange reaction not in model: R_EX_rbflvrd_e
  warn(f'Exchange reaction not in model: {r_id}')
/Users/idunmariaburgos/opt/anaconda3/envs/MAGs_RecAndAn2/lib/python3.9/site-packages/reframed/core/environment.py:75: UserWarning: Exchange reaction not in model: R_EX_tungs_e
  warn(f'Exchange reaction not in model: {r_id}')


		lac__L
		ac
		ppa
		but
		ibt
		pta
		ival
		hxa
		isocap
	Xylan
		etoh
		lac__L
		ac
		ppa
		but
		ibt
		pta
		ival
		hxa
		isocap
CD_P
	PASC
		etoh
		lac__L
		ac
		ppa
		but
		ibt
		pta
		ival
		hxa
		isocap
	Xylan
		etoh
		lac__L
		ac
		ppa
		but
		ibt
		pta
		ival
		hxa
		isocap
CD_X
	PASC
		etoh


/Users/idunmariaburgos/opt/anaconda3/envs/MAGs_RecAndAn2/lib/python3.9/site-packages/reframed/core/environment.py:75: UserWarning: Exchange reaction not in model: R_EX_pydxn_e
  warn(f'Exchange reaction not in model: {r_id}')
/Users/idunmariaburgos/opt/anaconda3/envs/MAGs_RecAndAn2/lib/python3.9/site-packages/reframed/core/environment.py:75: UserWarning: Exchange reaction not in model: R_EX_4abz_e
  warn(f'Exchange reaction not in model: {r_id}')
/Users/idunmariaburgos/opt/anaconda3/envs/MAGs_RecAndAn2/lib/python3.9/site-packages/reframed/core/environment.py:75: UserWarning: Exchange reaction not in model: R_EX_ascb__L_e
  warn(f'Exchange reaction not in model: {r_id}')
/Users/idunmariaburgos/opt/anaconda3/envs/MAGs_RecAndAn2/lib/python3.9/site-packages/reframed/solvers/gurobi_solver.py:236: UserWarning: Constrained variable 'R_EX_ascb__L_e' not previously declared
  warn(f"Constrained variable '{r_id}' not previously declared")


		lac__L
		ac
		ppa
		but
		ibt
		pta
		ival
		hxa
		isocap
	Xylan
		etoh


...

		ac
		ppa
		but
		ibt
		pta
		ival
		hxa
		isocap
	Xylan
		etoh
		lac__L
		ac
		ppa
		but
		ibt
		pta
		ival
		hxa
		isocap
CM_P
	PASC
		etoh
		lac__L
		ac
		ppa
		but
		ibt
		pta
		ival
		hxa
		isocap
	Xylan
		etoh
		lac__L
		ac
		ppa
		but
		ibt
		pta
		ival
		hxa
		isocap
CM_X
	PASC
		etoh
		lac__L
		ac
		ppa
		but
		ibt
		pta
		ival
		hxa
		isocap
	Xylan
		etoh


/Users/idunmariaburgos/opt/anaconda3/envs/MAGs_RecAndAn2/lib/python3.9/site-packages/reframed/core/environment.py:75: UserWarning: Exchange reaction not in model: R_EX_xylan8_e
  warn(f'Exchange reaction not in model: {r_id}')


		lac__L
		ac
		ppa
		but
		ibt
		pta
		ival
		hxa
		isocap
M_P
	PASC
		etoh
		lac__L
		ac
		ppa
		but
		ibt
		pta
		ival
		hxa
		isocap
	Xylan
		etoh
		lac__L
		ac
		ppa
		but
		ibt
		pta
		ival
		hxa
		isocap
M_X
	PASC
		etoh


/Users/idunmariaburgos/opt/anaconda3/envs/MAGs_RecAndAn2/lib/python3.9/site-packages/reframed/core/environment.py:75: UserWarning: Exchange reaction not in model: R_EX_nac_e
  warn(f'Exchange reaction not in model: {r_id}')


		lac__L
		ac
		ppa
		but
		ibt
		pta
		ival
		hxa
		isocap
	Xylan
		etoh
		lac__L
		ac
		ppa
		but
		ibt
		pta
		ival
		hxa
		isocap


In [91]:
pd.DataFrame(pFBA_solutions)

CD_A                                \
                                       PASC                         Xylan   
carbon_usable  [cellb, cell3, cell4, cell5]  [xylb, xyl3, xylan4, xylan8]   
etoh                                   20.0                     20.228325   
lac__L                            15.879464                     14.054173   
ac                                23.034176                     24.353147   
ppa                               12.248737                     15.656788   
but                                    12.0                     11.908643   
ibt                                     0.0                           0.0   
pta                                  8.5801                      8.371664   
hxa                                     7.5                      7.542153   
isocap                                  7.5                      7.107846   
ival                                    NaN                           NaN   

                                       CD_P                                \
                                       PASC                         Xylan   
carbon_usable  [cellb, cell3, cell4, cell5]  [xylb, xyl3, xylan4, xylan8]   
etoh                              21.092004                     20.331786   
lac__L                            17.234478                     13.218567   
ac                                23.880952                     20.997481   
ppa                               17.773718                     16.689475   
but                               13.025518                     12.381228   
ibt                               12.326574                     11.280566   
pta                                9.704566                      9.128388   
hxa                                8.080126                      7.712257   
isocap                             7.884007                      7.377778   
ival                                    NaN                           NaN   

                                       CD_X                    \
                                       PASC             Xylan   
carbon_usable  [cellb, cell3, cell4, cell5]  [xylan4, xylan8]   
etoh                                   20.0         17.692308   
lac__L                             9.626511          7.369727   
ac                                20.895833         17.692308   
ppa                                8.309886          6.923077   
but                               10.534722          8.846154   
ibt                                    10.0              10.0   
pta                                5.558105          4.153846   
hxa                                7.023148          6.666667   
isocap                                  7.5          7.211538   
ival                                    NaN               NaN   

                                       CM_A                                \
                                       PASC                         Xylan   
carbon_usable  [cellb, cell3, cell4, cell5]  [xylb, xyl3, xylan4, xylan8]   
etoh                                    NaN                           NaN   
lac__L                                 20.0                     19.174411   
ac                                22.492547                     24.313535   
ppa                               17.142857                     15.159156   
but                               11.950617                     11.321789   
ibt                                    10.0                          10.0   
pta                                9.230769                      8.509197   
hxa                                     7.5                      5.619545   
isocap                                  NaN                           NaN   
ival                              17.142857                     15.571512   

                                       CM_P                                \
                                       PASC                         Xylan   
carbon_usable  [cellb, cell3, cel

**Testing for all reactions**